# Imports

In [74]:
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
import requests
import numpy as np
import pandas as pd
import requests
import time
import random
import math
from retrying import retry
from scipy.stats import norm, zscore
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
import statsmodels.api as sm
import scipy.stats as st
from statsmodels.tools import add_constant as add_constant
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.decomposition import PCA


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

# Helper Functions

In [6]:
def get_injury_data(url):
    print('here')
    response = requests.get(url)
    soup = soup_1(response.content, 'html.parser')
    # Locate the table containing the injury data
    injury_table = soup.find('table', {'id': 'team_injuries'})
    if injury_table:
        # Read the table using Pandas
        injuries_df = pd.read_html(str(injury_table))[0]
        
        print(injuries_df)
        # Clean and process the data if needed
        return injuries_df
    else:
        return None

def get_vegas_lines_data(url):
    response = requests.get(url)
    soup = soup_1(response.content, 'html.parser')
    # Locate the table containing the Vegas Lines data
    vegas_lines_table = soup.find('table', {'id': 'vegas_lines'})
    if vegas_lines_table:
        # Read the table using Pandas
        vegas_lines_df = pd.read_html(str(vegas_lines_table))[0]

        # Clean and process the data if needed
        return vegas_lines_df
    else:
        return None

from bs4 import BeautifulSoup
import requests

@retry(stop_max_attempt_number=4, wait_fixed=2000)
def team_has_data(team_abbreviation, season):
    print("TEAM:", team_abbreviation)
    time.sleep(random.randint(3,4))

    url = f'https://www.basketball-reference.com/teams/{team_abbreviation}/{season}/gamelog-advanced'
    response = requests.get(url)
    
    if response.status_code != 200:
        print(response.status_code)
        return False  # URL doesn't exist or couldn't be accessed
    
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'tgl_advanced'})
    return table is not None


def get_latest_week( specific_season, data):
    nba_df = pd.DataFrame()

    for season in range(specific_season, specific_season+1):
        for a in abbreviations:
            if not team_has_data(a, season):
                print(team_has_data(a,season))
                continue
            else:
                print('here')
                url = f'https://www.basketball-reference.com/teams/{a}/{season}/gamelog-advanced'
                injury_url = f'https://www.basketball-reference.com/teams/{a}/{season}.html#all_injuries'
                salary_url = f'https://www.basketball-reference.com/teams/{a}/{season}.html'


                time.sleep(random.randint(5,6))
                team_df = pd.read_html(url, header=1, attrs={'id': 'tgl_advanced'} )[0]
                #             def_df = pd.read_html(url, header=1, attrs={'id': 'gamelog_opp' + f'{season}'} )[0]

    #             def_df.columns =def_df.columns + '_2'
    #             team_df = pd.concat([team_df, def_df], axis=1)
                team_df.insert(loc=0, column='Season', value=season +1)
                team_df.insert(loc=2, column='Team', value=a.upper())
            
                team_df['Rk'] = pd.to_numeric(team_df['Rk'], errors='coerce')
                latest_game_number = int(team_df['Rk'].max())
                
                data['Rk'] = pd.to_numeric(data['Rk'], errors='coerce')
                filtered_data = data.loc[(data['Season'] == specific_season+1) & (data['Team'] == a)]
                
                
                current_game_number = int(filtered_data['Rk'].max())
                
                latest_games= pd.DataFrame()
                print(current_game_number,'Current Game', latest_game_number, 'Latest_Game')
                if current_game_number == latest_game_number:
                    print('Games already updated : Continuing')
                    continue
                print("Still going?")
                for game in range(current_game_number+1, latest_game_number+1):
                    latest_games = pd.concat([latest_games, team_df[team_df['Rk'] == game]])

                team_df = latest_games
                
                
                
                try:
                # Fetch the salary page
                    time.sleep(random.randint(3,5))

    #                 response = requests.get(salary_url)
    #                 response.raise_for_status()  # Check for request errors

                    # Parse the tables in the salary page
                    all_tables = pd.read_html(salary_url, header=0)

                    # Filter the salary table by its id 'salaries2'
                    experience =all_tables[0]['Exp']
                    experience = experience.replace({'R': 1, 'TBD': 1, '': 0}, regex=True)
                    experience = experience.fillna(0).astype(int)
                    total_experience = experience.sum()
                    team_df['Experience'] = total_experience
    #                 print(team_df['Experience'])
                
                except Exception as e:
                    print(f"Error fetching experience table for {a} in {season}: {e}")
                    team_df['Experience'] = np.nan
                    
                    
                try:
                    response = requests.get(salary_url)
                    page = re.sub("<!--|-->","",response.text)

                    soup = BeautifulSoup(page, 'html.parser')
    #                 salary = pd.read_html(salary_url, header=0 )
    #                 print(soup.prettify())

    #                 print(salary)
    #                 salary_table = soup2.find('table', {'id': 'salaries2'})
                    # Fetch the salary page
                    time.sleep(random.randint(3,5))



                    # Parse the tables in the salary page
                    all_tables = pd.read_html(page, header=0)

                    # Filter the salary table by its id 'salaries2'
                    salary = all_tables[-1]['Salary'].replace('[\$,]', '', regex=True).fillna(0).astype(int)
                    salary = salary.sum()
                    team_df['Total_Salary'] = salary
                
                except Exception as e:
                    print(f"Error fetching salary table for {a} in {season}: {e}")
                    team_df['Total_Salary'] = np.nan
                
                nba_df = pd.concat([nba_df, team_df], ignore_index=True)
                nba_df['Experience'] = nba_df.groupby(['Season', 'Team_Code'])['Experience'].apply(lambda x: x.fillna(method='ffill'))
                nba_df['Total_Salary'] = nba_df.groupby(['Season', 'Team_Code'])['Total_Salary'].apply(lambda x: x.fillna(method='ffill'))

    
    data = pd.concat([data, nba_df], ignore_index=True)
    data.to_csv('nba_data2', index=False)
                

def convert_date(date_str):
    Month, Day = date_str.split()
    month_number = month_mapping.get(Month, None)
    
    date = f'{month_number}-{int(Day):02d}'
    print(date)
    return date

    


In [75]:
def rolling_averages_OPP(group, cols, new_cols):
        group = group.sort_values("Date")
        rolling_stats = group[cols].rolling(15, closed='left').mean()
        group[new_cols] = rolling_stats
        season = int(group['Season'][:1])
        code = float(group['Opp_Code'][:1])
        previous=nba_df[(nba_df['Season'] == (season-1)) & (nba_df['Opp_Code'] == code)]
        previous = previous.sort_values("Date")

        try:
            for a,b in zip(cols,new_cols):
                previous_rolling= previous[a].rolling(25,closed='right').mean()
    
                group[b].fillna(previous_rolling.iloc[-1], inplace=True)
        except:
            print("Didn't make it. ")
        
        return group
            

def rolling_averages_TM(group, cols, new_cols):
        group = group.sort_values("Date")
        rolling_stats = group[cols].rolling(15, closed='left').mean()
        group[new_cols] = rolling_stats
        season = int(group['Season'][:1])
        code = float(group['Team_Code'][:1])
        previous=nba_df[(nba_df['Season'] == (season-1)) & (nba_df['Team_Code'] == code)]
        previous = previous.sort_values("Date")

        try:
            for a,b in zip(cols,new_cols):
                previous_rolling= previous[a].rolling(25,closed='right').mean()
    
                group[b].fillna(previous_rolling.iloc[-1], inplace=True)
        except:
            print("Didn't make it. ")

        return group

# Scrape Nba Data from 1985 to 2024

In [ ]:
nba_df = pd.DataFrame()
import re

for season in range(1985, 2025):
    for a in abbreviations:
        if not team_has_data(a, season):
            print(team_has_data(a,season))
            continue
        else:
            print('here')
            url = f'https://www.basketball-reference.com/teams/{a}/{season}/gamelog-advanced'
            injury_url = f'https://www.basketball-reference.com/teams/{a}/{season}.html#all_injuries'
            salary_url = f'https://www.basketball-reference.com/teams/{a}/{season}.html'
            
            
            time.sleep(random.randint(3,6))
            team_df = pd.read_html(url, header=1, attrs={'id': 'tgl_advanced'} )[0]
            #             def_df = pd.read_html(url, header=1, attrs={'id': 'gamelog_opp' + f'{season}'} )[0]
            
#             def_df.columns =def_df.columns + '_2'
#             team_df = pd.concat([team_df, def_df], axis=1)
            team_df.insert(loc=0, column='Season', value=season +1)
            team_df.insert(loc=2, column='Team', value=a.upper())
            print(salary_url)
            try:
                # Fetch the salary page
                time.sleep(random.randint(3,5))

#                 response = requests.get(salary_url)
#                 response.raise_for_status()  # Check for request errors

                # Parse the tables in the salary page
                all_tables = pd.read_html(salary_url, header=0)
                
                # Filter the salary table by its id 'salaries2'
                experience =all_tables[0]['Exp']
                experience = experience.replace({'R': 1, 'TBD': 1, '': 0}, regex=True)
                experience = experience.fillna(0).astype(int)
                total_experience = experience.sum()
                team_df['Experience'] = total_experience
#                 print(team_df['Experience'])
                
            except Exception as e:
                print(f"Error fetching experience table for {a} in {season}: {e}")
                team_df['Experience'] = 0
                
                
            try:
                response = requests.get(salary_url)
                page = re.sub("<!--|-->","",response.text)

                soup = BeautifulSoup(page, 'html.parser')
#                 salary = pd.read_html(salary_url, header=0 )
#                 print(soup.prettify())
                
#                 print(salary)
#                 salary_table = soup2.find('table', {'id': 'salaries2'})
                # Fetch the salary page
                time.sleep(random.randint(3,5))

                

                # Parse the tables in the salary page
                all_tables = pd.read_html(page, header=0)
                
                # Filter the salary table by its id 'salaries2'
                salary = all_tables[-1]['Salary'].replace('[\$,]', '', regex=True).fillna(0).astype(int)
                salary = salary.sum()
                team_df['Total_Salary'] = salary
                
            except Exception as e:
                print(f"Error fetching salary table for {a} in {season}: {e}")
                team_df['Total_Salary'] = 0
                
            
                
            nba_df = pd.concat([nba_df, team_df], ignore_index=True)
            print(url)
nba_df.to_csv('nba_data_SalaryEXP1985', index=False)

# Begin Data Cleaning

In [76]:
nba_df = pd.read_csv('nba_data_SalaryEXP1985')


nba_df['Season'] = nba_df['Season'] -1
nba_df.rename(columns= {'eFG%.1': 'OppeFG%', 'TOV%.1': 'OppTOV%', 'FT/FGA.1': 'OppFT/FGA', 'Unnamed: 3': 'Home/Away',}, inplace=True)
nba_df.rename(columns={'Unnamed: 3': 'Home/Away'}, inplace =True)
threshold = len(nba_df.columns) * 0.75
nba_df = nba_df.dropna(thresh=threshold)
nba_df.drop(columns=['Rk', 'Unnamed: 18', 'Unnamed: 23'], inplace=True)


# Made Team Codes for each team and Opponents to track stats across games and seasons.

In [77]:
mapped_abbreviations = {
    'ATL' : 0, 'BOS' : 1, 'BRK' : 2, 'CHO': 3, 'CHI': 4, 'CLE': 5, 'DAL': 6, 'DEN': 7, 'DET':8, 'GSW':9, 'HOU':10, 
    'IND':11, 'LAC':12, 'LAL':13, 'MEM':14, 'MIA':15, 'MIL':16, 'MIN':17, 'NOP':18, 'NYK':19, 'OKC':20, 'ORL':21, 
    'PHI':22, 'PHO':23, 'POR':24, 'SAC':25, 'SAS':26, 'TOR':27, 'UTA':28, 'WAS':29, 'VAN': 14,'NJN': 2, 'CHA':3, 'SEA': 20
    ,'NOH': 18, 'NOK': 3, 'CHH': 3, 'WSB' :29, 'KCK':25,'SDC':12,'NOJ':28
}

nba_df['Team_Code'] = nba_df['Team'].map(mapped_abbreviations)
nba_df['Opp_Code'] = nba_df['Opp'].map(mapped_abbreviations)



# Preparing my Dataset for rolling statistics

In [78]:
column_move = nba_df.pop('Team_Code')
nba_df.insert(2, "Team_Code", column_move)
column_move = nba_df.pop('Opp_Code')
nba_df.insert(5, "Opp_Code", column_move)

cols = ['Tm', 'Opp.1', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 
    'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'OppeFG%', 'OppTOV%', 
    'DRB%', 'OppFT/FGA'
]

nba_df[cols] = nba_df[cols].apply(pd.to_numeric, errors='coerce')
nba_df['eDIFF'] = nba_df['ORtg'] - nba_df['DRtg']
nba_df['pDIFF'] = nba_df['Tm'] -nba_df['Opp.1']
cols = ['Tm', 'Opp.1', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 
    'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'OppeFG%', 'OppTOV%', 
    'DRB%', 'OppFT/FGA','eDIFF', 'pDIFF'
]

new_cols = [f'{c}_rolling' for c in cols]
new_longer_cols = [f'{c}_Off_rolling' for c in cols]

Opp_cols = ['Tm', 'Opp.1', 'ORtg', 'DRtg', 'eFG%', 'TOV%',  'FT/FGA', 'OppeFG%', 'OppTOV%', 
     'OppFT/FGA', 'eDIFF', 'pDIFF']


new_opp_cols = ['OppPoints_Allowed_rolling', 'OppPoints_Scored_rolling', 'OppDRtg_rolling', 'OppORtg_rolling','DefenisveeFG%_rolling'
                , 'DefensiveTOV%_rolling', 'OppOppFT/FGA_rolling','OppOppeFG_rolling','OppOppTOV%_rolling', 'Opp2FT/FGA_rolling', 'OppeDIFF_rolling', 'OpppDIFF_rolling']

new_opp_longer_cols = [f'{c}_Def_longer' for c in new_opp_cols]
nba_df[cols] = nba_df[cols].apply(pd.to_numeric, errors='coerce')

# Rolling statistics of features

In [79]:
OppNba_df = nba_df.groupby(["Season","Opp_Code"]).apply(lambda x: rolling_averages_OPP(x, Opp_cols, new_opp_cols))

OppNba_df.index = range(OppNba_df.shape[0])

Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 


In [80]:
TmNba_df = nba_df.groupby(["Season","Team_Code"]).apply(lambda x: rolling_averages_TM(x, cols, new_cols))
TmNba_df.index = range(TmNba_df.shape[0])


Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 
Didn't make it. 


In [81]:
TmNba_df = TmNba_df.dropna(subset=new_cols)
OppNba_df = OppNba_df.dropna(subset=new_opp_cols)


# Merge the datasets back together and further cleaning.

In [82]:
Merged_nba_df = pd.merge(TmNba_df, OppNba_df, on=['Season', 'G', 'Team_Code'])
columns_to_drop = [col for col in Merged_nba_df.columns if col.endswith('_y')]
Merged_nba_df = Merged_nba_df.drop(columns=columns_to_drop)
Merged_nba_df.columns= Merged_nba_df.columns.str.replace('_x', '')
Merged_nba_df['Home/Away'].fillna(1, inplace=True)
Merged_nba_df['Home/Away'].replace('@', 0, inplace=True)
Merged_nba_df['Date'] = pd.to_datetime(Merged_nba_df['Date'])
Merged_nba_df['Opp_Code'] = Merged_nba_df['Opp_Code'].astype(int)
Merged_nba_df['Unique_ID'] = Merged_nba_df.apply(lambda row: f"{row['Season']}_{row['G']}_{row['Team_Code']}@{row['Opp_Code']}" if row['Home/Away'] == 0 else f"{row['Season']}_{row['G']}_{row['Opp_Code']}@{row['Team_Code']}", axis=1)
Merged_nba_df.sort_values(by='Date', inplace=True)
Merged_nba_df['Total_Points'] = Merged_nba_df['Tm'] + Merged_nba_df['Opp.1']
Merged_nba_df['OppeDIFF_rolling'] = -Merged_nba_df['OppeDIFF_rolling']
Merged_nba_df['OpppDIFF_rolling'] = -Merged_nba_df['OpppDIFF_rolling']
Merged_nba_df['G'] = Merged_nba_df['G'].astype(int)
duplicates = Merged_nba_df.duplicated(subset=['Unique_ID'], keep="first")


# Making Targets for Predictions

In [83]:
Merged_nba_df['W/L'].replace('W', 1, inplace =True)
Merged_nba_df['W/L'].replace('L', 0, inplace =True)

In [84]:
Merged_nba_df['Date'] = pd.to_datetime(Merged_nba_df['Date'])


# Feature Engineering

In [85]:
Merged_nba_df['Rest_Days'] = Merged_nba_df.groupby(["Season", "Team_Code"])['Date'].diff().dt.days
Merged_nba_df['Rest_Days'].clip(lower=0, inplace=True)
Merged_nba_df['Rest_Days'].fillna(0,inplace=True)
Merged_nba_df['Rest_Days'].clip(upper=9, inplace=True)

In [86]:
Merged_nba_df['Opp_Rest_Days'] = Merged_nba_df.groupby(['Season', 'Opp_Code'])['Date'].diff().dt.days
Merged_nba_df['Opp_Rest_Days'].clip(lower=0, inplace=True)
Merged_nba_df['Opp_Rest_Days'].fillna(0,inplace=True)
Merged_nba_df['Opp_Rest_Days'].clip(upper=9, inplace=True)

In [87]:
Merged_nba_df['TmeDIFFProduct'] = Merged_nba_df['eDIFF_rolling'] * Merged_nba_df['Tm_rolling']
Merged_nba_df['TmpDIFFProduct'] = Merged_nba_df['pDIFF_rolling'] * Merged_nba_df['Tm_rolling']
Merged_nba_df['TmOppAllowedProduct'] = Merged_nba_df['Tm_rolling'] * Merged_nba_df['OppPoints_Allowed_rolling']
Merged_nba_df['PaceTmProduct'] = Merged_nba_df['Pace_rolling'] * Merged_nba_df['Tm_rolling']


In [88]:
Merged_nba_df['Expanding_Total_Points'] = Merged_nba_df.groupby(['Season', 'Team_Code'])['Total_Points'].apply(lambda x: x.shift(1).expanding(min_periods=1).mean()).reset_index(level=[0, 1], drop=True)
Merged_nba_df['Expanding_Opp_Total_Points'] = Merged_nba_df.groupby(['Season', 'Opp_Code'])['Total_Points'].apply(lambda x: x.shift(1).expanding(min_periods=1).mean()).reset_index(level=[0, 1], drop=True)
Merged_nba_df['Expanding_Total_Points'].fillna(0,inplace=True)
Merged_nba_df['Expanding_Opp_Total_Points'].fillna(0,inplace=True)


# Identifying Predictors

In [89]:
predictors = [ 'Season',
              'G',
              'Team_Code',
              'Opp_Code',
              'Home/Away',
              'Tm_rolling',
              'Opp.1_rolling',
              'ORtg_rolling',
              'DRtg_rolling',
              'Pace_rolling',
              'FTr_rolling',
              '3PAr_rolling',
              'TS%_rolling',
              'TRB%_rolling',
              'AST%_rolling',
              'eFG%_rolling',
              'TOV%_rolling',
              'ORB%_rolling',
              'FT/FGA_rolling',
              'OppeFG%_rolling',
              'OppTOV%_rolling',
              'DRB%_rolling',
              'OppFT/FGA_rolling',
              'OppPoints_Allowed_rolling',
              'OppPoints_Scored_rolling',
              'OppDRtg_rolling',
              'OppORtg_rolling',
              'DefenisveeFG%_rolling',
              'DefensiveTOV%_rolling',
              'OppOppFT/FGA_rolling',
              'OppOppeFG_rolling',
              'OppOppTOV%_rolling',
              'Opp2FT/FGA_rolling',
              'eDIFF_rolling',
    'OppeDIFF_rolling',
                 'pDIFF_rolling',
    'OpppDIFF_rolling',
                 'TmeDIFFProduct',
    'TmpDIFFProduct',
    'TmOppAllowedProduct',
              "PaceTmProduct",
                  "Rest_Days",
    "Opp_Rest_Days",
    "Experience",
              "Total_Salary",              
              'Expanding_Total_Points','Expanding_Opp_Total_Points'
              
             ]

In [90]:
Merged_nba_df[predictors] = Merged_nba_df[predictors].apply(pd.to_numeric, errors='coerce')

In [91]:
Merged_nba_df['W/L'].dtypes

dtype('int64')

# For logistic regression I want to find all features that have significant p-values through backwards elimination 

In [97]:


X_logistic = Merged_nba_df[predictors]
X_constant = add_constant(X_logistic)
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X_constant)
X_scaled = pd.DataFrame(X_scaled,columns=X_constant.columns, index= X_logistic.index)
def backward_elimination(X_data,target, col_list):
    
    while len(col_list) > 0:
        model = sm.Logit(target,X_data[col_list])
        result = model.fit(disp=0)
        largest_pvalue = round(result.pvalues, 3).nlargest(1)
        if largest_pvalue[0] <(0.05):
            return result, col_list
            break
        else:
            col_list.remove(largest_pvalue.index)
            
result, optimal_columns = backward_elimination(X_constant,Merged_nba_df['W/L'], predictors)


In [98]:
optimal_columns

['Season',
 'Home/Away',
 'Tm_rolling',
 'DRtg_rolling',
 'FTr_rolling',
 'TS%_rolling',
 'AST%_rolling',
 'ORB%_rolling',
 'OppeFG%_rolling',
 'OppTOV%_rolling',
 'DRB%_rolling',
 'OppFT/FGA_rolling',
 'OppPoints_Scored_rolling',
 'OppORtg_rolling',
 'DefenisveeFG%_rolling',
 'DefensiveTOV%_rolling',
 'OppOppeFG_rolling',
 'OppOppTOV%_rolling',
 'Opp2FT/FGA_rolling',
 'pDIFF_rolling',
 'OpppDIFF_rolling',
 'TmpDIFFProduct',
 'TmOppAllowedProduct',
 'Rest_Days',
 'Opp_Rest_Days',
 'Experience',
 'Total_Salary']

# Finally we can predict a win or loss using our dataset.

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.decomposition import PCA

import numpy as np

# Split the data into training, validation, and testing sets based on the 'Season' column
logits_train = Merged_nba_df[Merged_nba_df['Season'] <= 2008]
logits_val = Merged_nba_df[(Merged_nba_df['Season'] > 2008) & (Merged_nba_df['Season'] <= 2010)]
logits_test = Merged_nba_df[Merged_nba_df['Season'] > 2010]

# Define predictors and target variable
X_logits_train = logits_train[optimal_columns]
logits_y_train = logits_train['W/L']
X_logits_val = logits_val[optimal_columns]
logits_y_val = logits_val['W/L']
X_logits_test = logits_test[optimal_columns]
logits_y_test = logits_test['W/L']

# Standardize the features
scaler_logits = StandardScaler()
X_logits_train_scaled = scaler_logits.fit_transform(X_logits_train)
X_logits_val_scaled = scaler_logits.transform(X_logits_val)
X_logits_test_scaled = scaler_logits.transform(X_logits_test)

# pca = PCA(n_components=3).fit(X_logits_train_scaled)
# X_logits_train_PCA = pca.transform(X_logits_train_scaled)
# X_logits_test_PCA = pca.transform(X_logits_test_scaled)
# X_logits_val_PCA = pca.transform(X_logits_val_scaled)



# Initialize and train the logistic regression model
lclf = LogisticRegression(random_state=0, max_iter=1000,solver='liblinear', penalty="l2", C=5.0)
lclf.fit(X_logits_train_scaled, logits_y_train)

# Predict and evaluate on the validation set
y_val_pred = lclf.predict(X_logits_val_scaled)
y_val_prob = lclf.predict_proba(X_logits_val_scaled)

# Predict and evaluate on the test set
y_test_pred = lclf.predict(X_logits_test_scaled)
y_test_prob = lclf.predict_proba(X_logits_test_scaled)

# Predict and evaluate on the training set
train_y_pred = lclf.predict(X_logits_train_scaled)
train_y_prob = lclf.predict_proba(X_logits_train_scaled)

# Calculate accuracies
val_accuracy = lclf.score(X_logits_val_scaled, logits_y_val)
print("Validation Accuracy: %.3f%%" % (val_accuracy * 100.0))

test_accuracy = lclf.score(X_logits_test_scaled, logits_y_test)
print("Test Accuracy: %.3f%%" % (test_accuracy * 100.0))

train_accuracy = lclf.score(X_logits_train_scaled, logits_y_train)
print("Train Accuracy: %.3f%%" % (train_accuracy * 100.0))

# Calculate ROC AUC scores
roc_auc_val = roc_auc_score(logits_y_val, y_val_prob[:, 1])
print("Validation ROC AUC: %.3f" % roc_auc_val)

roc_auc_test = roc_auc_score(logits_y_test, y_test_prob[:, 1])
print("Test ROC AUC: %.3f" % roc_auc_test)

roc_auc_train = roc_auc_score(logits_y_train, train_y_prob[:, 1])
print("Train ROC AUC: %.3f" % roc_auc_train)

# Print classification report for validation and test sets
print("\nValidation Classification Report:\n", classification_report(logits_y_val, y_val_pred))
print("\nTest Classification Report:\n", classification_report(logits_y_test, y_test_pred))


Validation Accuracy: 67.836%
Test Accuracy: 64.784%
Train Accuracy: 68.102%
Validation ROC AUC: 0.743
Test ROC AUC: 0.706
Train ROC AUC: 0.747

Validation Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.68      0.68      2011
           1       0.68      0.67      0.68      2009

    accuracy                           0.68      4020
   macro avg       0.68      0.68      0.68      4020
weighted avg       0.68      0.68      0.68      4020


Test Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.63      0.64     13516
           1       0.64      0.67      0.65     13500

    accuracy                           0.65     27016
   macro avg       0.65      0.65      0.65     27016
weighted avg       0.65      0.65      0.65     27016



# Model ranges from 67% accuracy to 64%. Also has a good ROC_AUC.
# Lets try and identify a strategy if I only chose games to bet on when my model has 80% confidence or higher it has ~ 80% accuracy with calling games.

In [100]:
result_df = pd.DataFrame(X_logits_test, columns=X_logits_test.columns)
result_df['G'] = logits_test['G']
result_df['Tm'] = logits_test['Tm']
result_df['Opp.1'] = logits_test['Opp.1']
result_df['Team'] = logits_test['Team']
result_df['Opp'] = logits_test['Opp']





result_df['actual'] = logits_y_test
result_df['predicted'] = y_test_pred
result_df['predicted_prob'] = y_test_prob[:,1]  # Only one column for probability of the positive class
result_df['confidence'] = result_df['predicted_prob']
result_df['high_confidence'] = result_df['confidence'] >= .80

# Filter high confidence predictions
high_confidence_df = result_df[result_df['high_confidence']]
high_confidence_df['predicted'] = 1
# Count correct and incorrect high confidence predictions
high_confidence_correct = high_confidence_df[high_confidence_df['actual'] == high_confidence_df['predicted']].shape[0]
high_confidence_incorrect = high_confidence_df[high_confidence_df['actual'] != high_confidence_df['predicted']].shape[0]

print(f"High Confidence Correct Predictions: {high_confidence_correct}")
print(f"High Confidence Incorrect Predictions: {high_confidence_incorrect}")

High Confidence Correct Predictions: 2242
High Confidence Incorrect Predictions: 572
